In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import boto3
from botocore.client import Config
import os
import pandas as pd
from io import StringIO
import matplotlib.dates as dates
from datetime import datetime
import matplotlib.ticker as ticker
from random import shuffle
from time import time
from pandas.plotting import register_matplotlib_converters
%matplotlib notebook

os.chdir('C:\\Users\\A Sua\\Documents\\AWS\\credentials')
list_of_lines = []
with open('capstoner_key.txt') as fp:
    lines = fp.readlines()
    for line in lines:
        line = line.strip()
        list_of_lines.append(line)
    fp.close()


location = list_of_lines[0].split('=')[1]
AWS_ACCESS_KEY_ID = list_of_lines[1].split('=')[1]
AWS_SECRET_ACCESS_KEY = list_of_lines[2].split('=')[1]
os.chdir('C:\\Users\\A Sua\\Documents\\FIU\\CAP2020')

In [ ]:
register_matplotlib_converters()
%matplotlib notebook

In [ ]:
df_name_list = ['df_returns07_18', 'df_price07_18']
df_list = []

### import dataframes

In [ ]:
### BUCKETS & KEYS
BUCKET_NAME = 'capstonefiu2020-data'
key_location = 'data/processed'

### ACTIVATE CLIENT
client = boto3.client('s3', 
              aws_access_key_id = AWS_ACCESS_KEY_ID,
               aws_secret_access_key=AWS_SECRET_ACCESS_KEY)


for name in df_name_list:
    file_name = '{}.csv'.format(name)
    key_path = key_location + '/' + file_name

    ### INSTANTIATE DATAFRAME
    csv_obj = client.get_object(Bucket = BUCKET_NAME, Key=key_path)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    temp_df = pd.read_csv(StringIO(csv_string))
    temp_df.index = temp_df['Unnamed: 0']
    temp_df = temp_df.drop('Unnamed: 0', axis=1)
    temp_df.index.names = ['date']


    #df_bidlo.head()
    #### TRANSPOSE DATA FOR EASE OF USE
    temp_df = temp_df.T
    temp_df.index = pd.to_datetime(temp_df.index)
    df_list.append(temp_df)
    print('{} added to df_list'.format(name))

In [ ]:
df_returns = df_list[0]
df_price = df_list[1]

### Find the dates of the worst trading days systemically & by the company

In [ ]:
dates = df_price.index
df_returns.columns[:5]

#### RETURNS: 5 worst Return dates: DOW JONES, S&P

In [ ]:
worst_dates_sp = pd.to_datetime(['2008-10-15', '2008-09-29', '2008-12-01', '2008-10-09', '2008-11-20'])
worst_dates_sp

### RETURNS: 5 worst Return dates: COMPNAY

In [ ]:
worst_days_returns = {}
for symbol in df_returns.columns:
    temp_series = df_returns[symbol].sort_values(ascending=True).iloc[:5]
    worst_days_returns[symbol] = temp_series
worst_days_returns['ORCL_ORACLE CORP']

### Price: 5 worst Return dates: COMPNAY

In [ ]:
worst_days_prices = {}
for symbol in df_price:
    temp_dic = worst_days_returns[symbol]
    temp_series = pd.merge(temp_dic, df_price[symbol], left_index=True, right_index=True, copy=False, suffixes=('_x', ''))[symbol]
    worst_days_prices[symbol] = temp_series
worst_days_prices

### Price: 5 worst Return dates: S&P

In [ ]:
worst_days_prices_sp = {}
for symbol in df_price.columns:
    temp_series = pd.Series([df_price[symbol][worst_dates_sp[i]] for i in range(len(worst_dates_sp))], index=[worst_dates_sp])
    worst_days_prices_sp[symbol] = temp_series

worst_days_prices_sp['HON_HONEYWELL INTERNATIONAL INC']

### Grid Plot of Price for Each Company 384 Plots (16 graph per plot)
#### the leftovers are plotted in the leftovers

In [ ]:
#sup_title_font =15
sub_title_font =8
legend_font =8
axis_font = 6
tick_rotation = 20
black_dot = 2
red_dot = 5

### matrix of subplots
n_cols = 4
n_rows = 4
n_plots = n_cols*n_rows
nloops = int(np.floor(388/n_plots))
temp_matrix = np.array(range(n_plots)).reshape(n_rows,n_cols)

for i in range(nloops):
    nn = len(df_price.columns)
    lower = i*n_plots
    upper = (i+1)*n_plots
    #print(lower, upper)
    temp_list = df_price.columns[lower:upper]
    print('\n\n\n\n\n\n\n\nBatch {}'.format(i))
    
    
    ####----PLOT THE FIGURES---------#####
    ### legend labels
    dot_labels = ['Price', 'Company', 'Market']
    temp_list_of_axis = []
    
    ### instantiage
    fig, ax = plt.subplots(n_rows,n_cols,figsize=(17,9))
    for k in range(n_rows):
        for l in range(n_cols):
            ## indices
            temp_idx = temp_matrix[k,l]
            symbol = temp_list[temp_idx]
            ### company prices, worst 5 dates with worst returns
            temp_worst_prices = worst_days_prices[symbol]
            ### comapny returns, worst 5 systemic dates with worst returns
            temp_worst_prices_sp = worst_days_prices_sp[symbol]

            ### Company Timeseries            
            wt = ax[k, l].plot(dates, df_price[symbol].values, label='Daily Price'.format(symbol), alpha=0.7)            

            ### PLOT: 5 worst company dates
            wc = ax[k, l].scatter(temp_worst_prices.index, temp_worst_prices.values, 
                       color='red', 
                       linewidths=red_dot, 
                       edgecolor='red') 
                       #label='Worst Company Trading')
            ### PLOT: Returns on 5 worst systemic dates 
            wm = ax[k,l].scatter(temp_worst_prices_sp.index, temp_worst_prices_sp.values, 
                       color='black', 
                       linewidths=black_dot, 
                       edgecolor='black') 
                       #label='Worst Market Trading Days')            



            ax[k,l].spines['top'].set_visible(False)
            ax[k,l].spines['right'].set_visible(False)
            ax[k,l].set_title(symbol, fontsize=sub_title_font) 
            #ax[k,l].set_xlabel(fontsize=x_axis_font)
            ax[k,l].tick_params(axis="x", labelsize=axis_font, rotation=tick_rotation)
            ax[k,l].tick_params(axis="y", labelsize=axis_font)
            #ax[k,l].set_ylim([-0.45, 0.45])            
            temp_list_of_axis.append([wc, wm, wt])
         
            
            
    #fig.legend(bbox_to_anchor=(1.1, 1.1), prop={'size':15})
    #fig.suptitle('Daily Returns 2007-2018:\nBatch {}'.format(i), fontsize=sup_title_font)
    fig.subplots_adjust(hspace=0.7)
    leg = fig.legend(temp_list_of_axis, labels=dot_labels, 
               #title='Worst Trading Days', title_fontsize=25, 
               bbox_to_anchor=(1.0, 0.97), 
               prop={'size':legend_font})#, fontsize=sup_title_font)
    leg.set_in_layout(False)
    #plt.tight_layout(pad=0.5, w_pad=5.0, h_pad=0.0)
    plt.tight_layout()
    plt.show()
    #plt.savefig('plots\\timeseries\\price_timeseries_batch{}.png'.format(i+1), dpi=220) 
        
#             for tick in ax[k, l].get_xticklabels():
#                 tick.set_rotation(20)
#                 tick.

### Leftover plots

In [ ]:
#sup_title_font =15
sub_title_font =8
legend_font =8
axis_font = 6
tick_rotation = 20

### matrix of subplots
n_cols = 2
n_rows = 2
n_plots = n_cols*n_rows
nloops = 1
temp_matrix = np.array(range(n_plots)).reshape(n_rows,n_cols)

for i in range(nloops):
    nn = len(df_price.columns)
    #print(lower, upper)
    temp_list = df_returns.columns[-n_plots:]
    print('\n\n\n\n\n\n\n\nBatch {}'.format(i))
    
    
    ####----PLOT THE FIGURES---------#####
    dot_labels = ['Price', 'Company', 'Market']
    temp_list_of_axis = []
    
    fig, ax = plt.subplots(n_cols,n_rows,figsize=(17,9))
    for k in range(n_rows):
        for l in range(n_cols):
            ## indices
            temp_idx = temp_matrix[k,l]
            symbol = temp_list[temp_idx]
            ### company returns, worst 5 dates with worst returns
            temp_worst_price = worst_days_prices[symbol]
            ### comapny returns, worst 5 systemic dates with worst returns
            temp_worst_price_sys = worst_days_sys_prices[symbol]
            
            ### Company Timeseries            
            wt = ax[k, l].plot(dates, df_price[symbol].values, label='Daily Prices'.format(symbol), alpha=0.6)

            ### PLOT: 5 worst company dates
            wc = ax[k, l].scatter(temp_worst_price.index, temp_worst_price.values, 
                       color='red', 
                       linewidths=7, 
                       edgecolor='red', zorder=1) 
                       #label='Worst Company Trading')
            ### PLOT: Returns on 5 worst systemic dates 
            wm = ax[k,l].scatter(temp_worst_price_sys.index, temp_worst_price_sys.values, 
                       color='black', 
                       linewidths=3, 
                       edgecolor='black', zorder=1) 
                       #label='Worst Market Trading Days')            


            ax[k,l].spines['top'].set_visible(False)
            ax[k,l].spines['right'].set_visible(False)
            ax[k,l].set_title(symbol, fontsize=sub_title_font) 
            #ax[k,l].set_xlabel(fontsize=x_axis_font)
            ax[k,l].tick_params(axis="x", labelsize=axis_font, rotation=tick_rotation)
            #ax[k,l].set_ylim([-0.45, 0.3])
            ax[k,l].tick_params(axis="y", labelsize=axis_font)
            temp_list_of_axis.append([wt, wc, wm])
         
            
            
    #fig.legend(bbox_to_anchor=(1.1, 1.1), prop={'size':15})
    #fig.suptitle('Daily Returns 2007-2018:\nBatch {}'.format(i), fontsize=sup_title_font)
    fig.subplots_adjust(hspace=0.7)
    leg = fig.legend(temp_list_of_axis, labels=dot_labels, 
               #title='Worst Trading Days', title_fontsize=25, 
               bbox_to_anchor=(1.0, 0.97), 
               prop={'size':legend_font})#, fontsize=sup_title_font)
    leg.set_in_layout(False)
    #plt.tight_layout(pad=0.5, w_pad=5.0, h_pad=0.0)
    plt.tight_layout()
    plt.show()
    #plt.savefig('plots\\timeseries\\price_timeseries_batch{}.png'.format(i+1), dpi=220) 
    
    
#             for tick in ax[k, l].get_xticklabels():
#                 tick.set_rotation(20)
#                 tick.

In [ ]:
# ### company prices, worst 5 dates with worst returns
# temp_worst_prices = worst_days_prices['HON_HONEYWELL INTERNATIONAL INC']
# ### comapny returns, worst 5 systemic dates with worst returns
# temp_worst_prices_sys = worst_days_sys_prices['HON_HONEYWELL INTERNATIONAL INC']
# sub_title_font =8
# legend_font =8
# axis_font = 6
# tick_rotation = 20
# black_dot = 2
# red_dot = 5



# fig, ax = plt.subplots(1,1, figsize=(5,5))

# ### Company Timeseries            
# wt = ax.plot(dates, df_price['HON_HONEYWELL INTERNATIONAL INC'].values, label='Daily Price'.format(symbol), alpha=0.7)            

# ### PLOT: 5 worst company dates
# wc = ax.scatter(temp_worst_prices.index, temp_worst_prices.values, 
#            color='red', 
#            linewidths=red_dot, 
#            edgecolor='red') 
#            #label='Worst Company Trading')
# ### PLOT: Returns on 5 worst systemic dates 
# wm = ax.scatter(temp_worst_prices_sys.index, temp_worst_prices_sys.values, 
#            color='black', 
#            linewidths=black_dot, 
#            edgecolor='black') 
#            #label='Worst Market Trading Days')            



# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)
# ax.set_title(symbol, fontsize=sub_title_font) 
# #ax[k,l].set_xlabel(fontsize=x_axis_font)
# ax.tick_params(axis="x", labelsize=axis_font, rotation=tick_rotation)
# ax.tick_params(axis="y", labelsize=axis_font)
# # ax.set_ylim([-0.45, 0.45])            
# # temp_list_of_axis.append([wc, wm, wt])
# plt.show()